# Загрузка Pandas и очистка данных

In [ ]:
'''Здравствуйте, здесь в ноутбуке только предобработка, в каггле так же 
есть небольшой анализ распределения признаков. Сделал довольно мало, но в целом с работой разобрался.
Знаю, что можно было добавить еще очень много признаков, но я и так опоздал((. Дайте пж информацию по созвону, есть
много вопросов по проекту. Свой feature ingenering не объяснял, думаю, все и так понятно. Вообще работа очень интере
сная, намного интересней предыдущих проектов, и я бы к ней вернулся потом с новыми знаниями, чтобы улучшить пред
сказания.

В readme написал вопросы, прошу объяснить'''

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/kirill/Downloads/main_task_new.csv')

In [3]:
# функция для всех действий по предобработке
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].mean(), inplace=True)
    df_output['Cuisine Style'].fillna("[\'Vegetarian Friendly\']", inplace=True) #самая частая кухня
    df_output['Reviews'].fillna("[[], []]", inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output['City_copy'] = df_output['City'] #потребуется позже
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    # Создаем One-Hot Encoding для видов кухни в каждом ресторане
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda s: s[1:-1].split(', '))
    cuisines_set = set() # все виды кухонь
    for cuisines_list in df_output['Cuisine Style']:
        for cuisine in cuisines_list:
            cuisines_set.add(cuisine)
        
    for cuisine in cuisines_set: #создаем одноименные колонки
        df_output[cuisine] = [0 for i in range(0, 40000)]

    index = 0
    for cuisine_list in df_output['Cuisine Style']: # заполняем единицей если кухня представлена
        for cuisine in cuisine_list:
            df_output[cuisine][index] = 1
        index+=1
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    
    # Вычисляем к-во ресторанов для города в каждой строке
    restorants_in_city = df_output.groupby('City_copy')['Ranking'].count().to_dict()
    df_output['Restaurants Count'] = df_output['City_copy'].map(restorants_in_city)

    def get_Weighed_Rank_RK(row):
        '''
        Вычисляем относительную позицию ресторана среди всех ресторанов города
        '''
        Weighed_Rank = row['Ranking'] / row['Restaurants Count']

        return Weighed_Rank

    # Получаем относительную позицию ресторана среди всех ресторанов города
    df_output['Weighed Rank'] = df_output.apply(lambda x: get_Weighed_Rank_RK(x), axis=1)
    
    # создаем колонку численности города
    population = {
    'Paris': 2148327,
    'Stockholm': 1006984,
    'London': 8961989,
    'Berlin': 3644826,
    'Munich': 1471508,
    'Oporto': 287591,
    'Milan': 1366180,
    'Bratislava': 428672,
    'Vienna': 1911191,
    'Rome': 2870500,
    'Barcelona': 1664182,
    'Madrid': 3334730,
    'Dublin': 1173179,
    'Brussels': 185103,
    'Zurich': 434008,
    'Warsaw': 1702139,
    'Budapest': 1752286,
    'Copenhagen': 794128,
    'Amsterdam': 860124,
    'Lyon': 518635,
    'Hamburg': 1845229,
    'Lisbon': 506654,
    'Prague': 1335084,
    'Oslo': 693494,
    'Helsinki': 656611,
    'Edinburgh': 488050,
    'Geneva': 433978,
    'Ljubljana': 286745,
    'Athens': 664046,
    'Luxembourg': 124509,
    'Krakow': 779115  
    }
    df_output['Population'] = df_output.City_copy.apply(lambda x: population[x]/1000000)
    
    # создаем колонку разницы в днях между двумя отзывами
    date_list = []
    for string in df_output.Reviews:
        raw_s = string[string.rfind('['):-1]
        review_list = raw_s[1:-1].split(', ')
        if len(review_list)>1:
            date_list.append(review_list)
        else:
            date_list.append(0)
        
    list2=[]
    for pair in date_list:
        if pair==0:
            list2.append(0)
        else:
            list2.append((pd.to_datetime(pair[0])-pd.to_datetime(pair[1])).days)

    df_output['review_diff'] = list2
    
    # создаем колонку - ранг ценовой категории
    def price_lvl(s):
        if s=='$': 
            return 1
        elif s=='$$ - $$$': 
            return 2
        elif s=='$$$$': 
            return 3
        else:
            return 2
    
    df_output['Price_lvl'] = df_output['Price Range'].apply(price_lvl)
    
    # добавляем колонку - количество представленных видов кухни
    df_output['Cuisine_amount'] = df_output['Cuisine Style'].apply(lambda x: len(x))
    
    # добавляем информацию о пропущенных значениях
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Cuisine_Style_isNAN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Price_Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['review_existance'] = df_output.Reviews.apply(lambda x: 1 if x=='[[], []]' else 0)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    df_output = df_output.drop(['Cuisine Style', 'Price Range', 'Reviews', 'City_copy', 'URL_TA'], axis=1)
    
    return df_output

In [4]:
df = preproc_data(df)

<ipython-input-3-57f33d2c0cad>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output[cuisine][index] = 1


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [5]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop('Rating', axis = 1)
y = df['Rating']

In [6]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [7]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [8]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [9]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [10]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21090299999999998
